## Install and import required packages and libraries

In [1]:
pip install lxml html5lib beautifulsoup4 # Install required packages for webscraping

Note: you may need to restart the kernel to use updated packages.


In [2]:
!conda install -c conda-forge folium=0.5.0 --yes # Install required packages for folium

Solving environment: done

# All requested packages already installed.



In [3]:
!conda install -c conda-forge geopy --yes #Install required packages for geopy

Solving environment: done

# All requested packages already installed.



In [4]:
# library to handle data in a vectorized manner
import numpy as np 

# library for data analsysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# library to handle JSON files
import json 

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# library to handle requests
import requests

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium

# import url library
import urllib.request

print('Libraries imported.')

Libraries imported.


## Create Dataframe for List of Neighborhood in Singapore from downloaded dataset

In [5]:
df = pd.read_csv('/Users/alvon/Downloads/sg.csv')
df.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Singapore,1.3000,103.8000,Singapore,SG,Central Singapore,primary,5745000.0,5638700.0
1,Keat Hong Village,1.3778,103.7442,Singapore,SG,South West,NaN,NaN,NaN
2,Choa Chu Kang New Town,1.3833,103.7500,Singapore,SG,South West,NaN,NaN,NaN
3,Bukit Panjang Estate,1.3817,103.7525,Singapore,SG,South West,NaN,NaN,NaN
4,Jalan Kayu,1.3972,103.8719,Singapore,SG,Central Singapore,NaN,NaN,NaN


## Clean & Process the dataset

In [6]:
#Remove rows with City as Singapore since it is not meaningful
df = df.loc[df['city']!='Singapore']

#Reset index
df.reset_index(inplace = True, drop = True)
df.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Keat Hong Village,1.3778,103.7442,Singapore,SG,South West,NaN,NaN,NaN
1,Choa Chu Kang New Town,1.3833,103.7500,Singapore,SG,South West,NaN,NaN,NaN
2,Bukit Panjang Estate,1.3817,103.7525,Singapore,SG,South West,NaN,NaN,NaN
3,Jalan Kayu,1.3972,103.8719,Singapore,SG,Central Singapore,NaN,NaN,NaN
4,Bukit Sembawang Estate,1.3956,103.8692,Singapore,SG,Central Singapore,NaN,NaN,NaN


In [7]:
#Drop columns that are not useful
df = df[['admin_name', 'city', 'lat', 'lng']]
df.head()

,admin_name,city,lat,lng
0,South West,Keat Hong Village,1.3778,103.7442
1,South West,Choa Chu Kang New Town,1.3833,103.7500
2,South West,Bukit Panjang Estate,1.3817,103.7525
3,Central Singapore,Jalan Kayu,1.3972,103.8719
4,Central Singapore,Bukit Sembawang Estate,1.3956,103.8692


In [8]:
#Rename columns
df.rename(columns={'admin_name':'Area', 'city':'Neighborhood', 'lat':'Latitude', 'lng':'Longitude'}, inplace = True)
df.head()

,Area,Neighborhood,Latitude,Longitude
0,South West,Keat Hong Village,1.3778,103.7442
1,South West,Choa Chu Kang New Town,1.3833,103.7500
2,South West,Bukit Panjang Estate,1.3817,103.7525
3,Central Singapore,Jalan Kayu,1.3972,103.8719
4,Central Singapore,Bukit Sembawang Estate,1.3956,103.8692


In [9]:
#Find the shape of the dataframe
df.shape

(74, 4)

## Explore neighborhood data in Singapore

### Prepare FourSquare data

In [10]:
CLIENT_ID = '2TSRP3NYRV5C4TAJZOYTE0WZM5PE3C5COCSZT1XAFBW4MAV2'
CLIENT_SECRET = '2B3IUP3EOJ5DUNGUJRI4LWBD25SNQRVPMSRVD5CQMK2BUCX4'
VERSION = '20180605'
LIMIT = 100

### Define a function to get nearby venues for each neighbourhood in Singapore, with output set as Dataframe

In [11]:
#Create a function to repeat the same process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
Singapore_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Keat Hong Village
Choa Chu Kang New Town
Bukit Panjang Estate
Jalan Kayu
Bukit Sembawang Estate
Kampong Serangoon Kechil
Kampong Cutforth
Mok Peng Hiang Estate
Woodlands New Town
Kampong Pinang
Serangoon
Kampong Sungai Tengah
Kampong Tongkang Pechah
Kampong Teban
Kangkar
Jurong Town
Kampong Ulu Jurong
Seletar Hills Estate
Tay Keng Loon Estate
Matilda Estate
Bukit Panjang New Town
Kampong Beremban
Marsiling
Chia Keng
Somapah Serangoon
Jurong West
Boon Lay
Kampong Chai Chee
Ulu Bedok
Kampong Pachitan
Bedok New Town
Kampong Sireh
Bedok Ville
Kampong Bukit Panjang
Kampong Lew Lian
Marsiling Estate
Yio Chu Kang Estate
Yio Chu Kang
Ang Mo Kio New Town
Kampong Amoy Quee
Singapore United Plantation
Kampong Ladang
Kampong Ubi
Kampong Reteh
Kampong Kebun Baharu
Ong Lee
Saint Michael’s Estate
Teacher’s Housing Estate
Saga
Kampong Kopit
Tanjong Pagar
Nee Soon Estate
Kampong Pengkalan Petai
Kampong Pengkalan Kundor
Kampong Siren
Springleaf Park
Kampong Loyang
Kampong Sungai Blukar
Kam Wak Hassan
Su

### Check size of the resulting dataframe

In [13]:
print(Singapore_venues.shape)
Singapore_venues.head()

(1049, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Keat Hong Village,1.3778,103.7442,KFC,1.377227,103.744419,Fast Food Restaurant
1,Keat Hong Village,1.3778,103.7442,Keat Hong Shopping Centre,1.377293,103.744429,Miscellaneous Shop
2,Keat Hong Village,1.3778,103.7442,85 Cafe,1.377323,103.744407,Food Court
3,Keat Hong Village,1.3778,103.7442,Hong Kiat Seafood Restaurant,1.376468,103.743865,Coffee Shop
4,Keat Hong Village,1.3778,103.7442,SMRT Buses: Bus 172,1.379646,103.743492,Bus Line


### Check how many venues were returned for each neighborhood

In [14]:
venue_count = Singapore_venues.groupby('Neighborhood').count()
venue_count

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ang Mo Kio New Town,14,14,14,14,14,14
Bedok New Town,63,63,63,63,63,63
Bedok Ville,10,10,10,10,10,10
Boon Lay,3,3,3,3,3,3
Bukit Panjang Estate,7,7,7,7,7,7
Bukit Panjang New Town,50,50,50,50,50,50
Bukit Sembawang Estate,14,14,14,14,14,14
Chia Keng,27,27,27,27,27,27
Choa Chu Kang New Town,8,8,8,8,8,8


In [15]:
#Confirm number of neighborhood
print('venue_count: ', venue_count.shape)
print('df: ',df.shape)

venue_count:  (74, 6)
df:  (74, 4)


### Find out how many unique categories can be curated from all the returned venues

In [16]:
print('There are {} uniques categories.'.format(len(Singapore_venues['Venue Category'].unique())))

There are 192 uniques categories.


### Analyze Each Neighborhood

In [17]:
# one hot encoding
Singapore_onehot = pd.get_dummies(Singapore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Singapore_onehot.insert(0, 'Neighborhood', Singapore_venues['Neighborhood'], True)

Singapore_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Arcade,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Badminton Court,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Campground,Cantonese Restaurant,Cheese Shop,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Coffee Shop,College Auditorium,College Cafeteria,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Factory,Farm,Farmers Market,Fast Food Restaurant,Fishing Spot,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Halal Restaurant,Health Food Store,High School,Historic Site,Hong Kong Restaurant,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Laundromat,Light Rail Station,Lottery Retailer,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,New American Restaurant,Night Market,Noodle House,Office,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Wings Joint,Yoga Studio
0,Keat Hong Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Keat Hong Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Keat Hong Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Keat Hong Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
Singapore_grouped = Singapore_onehot.groupby('Neighborhood').mean().reset_index()
Singapore_grouped.head()

,Neighborhood,Accessories Store,American Restaurant,Arcade,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Badminton Court,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Bowling Green,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Campground,Cantonese Restaurant,Cheese Shop,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Coffee Shop,College Auditorium,College Cafeteria,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Factory,Farm,Farmers Market,Fast Food Restaurant,Fishing Spot,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden Center,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Halal Restaurant,Health Food Store,High School,Historic Site,Hong Kong Restaurant,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Laundromat,Light Rail Station,Lottery Retailer,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,New American Restaurant,Night Market,Noodle House,Office,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tennis Court,Thai Restaurant,Theater,Theme Park,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Wings Joint,Yoga Studio
0,Ang Mo Kio New Town,0.0,0.000000,0.0,0.0,0.000000,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.071429,0.0,0.000000,0.0,0.0,0.0,0.0,0.071429,0.000000,0.071429,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.285714,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.071429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.071429,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.071429,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.0,0.000000,0.0
1,Bedok New Town,0.0,0.031746,0.0,0.0,0.047619,0.0,0.000000,0.0,0.0,0.031746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.0,0.015873,0.0,0.015873,0.0,0.015873,0.015873,0.0,0.0,0.000000,0.0,0.047619,0.0,0.0,0.0,0.0,0.063492,0.015873,0.063492,

### Confirm the new size

In [19]:
Singapore_grouped.shape

(74, 193)

### Print each neighborhood along with the top 5 most common venues

In [20]:
num_top_venues = 5

for hood in Singapore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Singapore_grouped[Singapore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ang Mo Kio New Town----
                 venue  freq
0           Food Court  0.29
1         Noodle House  0.07
2  Shanghai Restaurant  0.07
3   Chinese Restaurant  0.07
4      Thai Restaurant  0.07


----Bedok New Town----
                venue  freq
0  Chinese Restaurant  0.06
1         Coffee Shop  0.06
2                Café  0.05
3    Asian Restaurant  0.05
4         Supermarket  0.05


----Bedok Ville----
              venue  freq
0    Ice Cream Shop   0.1
1             Trail   0.1
2       Coffee Shop   0.1
3        Food Court   0.1
4  Malay Restaurant   0.1


----Boon Lay----
               venue  freq
0        Coffee Shop  0.33
1        Music Venue  0.33
2               Café  0.33
3  Accessories Store  0.00
4       Night Market  0.00


----Bukit Panjang Estate----
                  venue  freq
0           Coffee Shop  0.29
1  Fast Food Restaurant  0.14
2     Convenience Store  0.14
3                  Café  0.14
4           Supermarket  0.14


----Bukit Panjang New Town----
  

### Define a function to sort the venues in descending order.

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create a new dataframe and display the top 10 venues for each neighborhood.

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Singapore_grouped['Neighborhood']

for ind in np.arange(Singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ang Mo Kio New Town,Food Court,Park,Australian Restaurant,Shanghai Restaurant,Bus Stop,Chinese Restaurant,Thai Restaurant,Coffee Shop,Malay Restaurant,Vegetarian / Vegan Restaurant
1,Bedok New Town,Chinese Restaurant,Coffee Shop,Sushi Restaurant,Café,Food Court,Asian Restaurant,Supermarket,Garden Center,Fried Chicken Joint,Fast Food Restaurant
2,Bedok Ville,Food Court,Dessert Shop,Malay Restaurant,Arts & Entertainment,Asian Restaurant,Halal Restaurant,Trail,Supermarket,Ice Cream Shop,Coffee Shop
3,Boon Lay,Café,Coffee Shop,Music Venue,Yoga Studio,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
4,Bukit Panjang Estate,Coffee Shop,Convenience Store,Fast Food Restaurant,Grocery Store,Supermarket,Café,Factory,Food Court,Food & Drink Shop,Food


### 3.2. Cluster Neighborhoods

In [23]:
### set number of clusters
kclusters = 4

Singapore_grouped_clustering = Singapore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Singapore_merged = df

# set Neighborhood as index in Singapore_merged dataframe
Singapore_merged.set_index('Neighborhood')

# merge with Singapore_merged to add latitude/longitude for each neighborhood
Singapore_merged = Singapore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# reset index
Singapore_merged.reset_index()

Singapore_merged

,Area,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,South West,Keat Hong Village,1.3778,103.7442,3,Miscellaneous Shop,Food Court,Coffee Shop,Bus Line,Fast Food Restaurant,Kids Store,Yoga Studio,Farmers Market,Food Stand,Food & Drink Shop
1,South West,Choa Chu Kang New Town,1.3833,103.7500,2,Coffee Shop,Road,Paper / Office Supplies Store,Fast Food Restaurant,Gym,Grocery Store,Café,Yoga Studio,Food & Drink Shop,Food
2,South West,Bukit Panjang Estate,1.3817,103.7525,2,Coffee Shop,Convenience Store,Fast Food Restaurant,Grocery Store,Supermarket,Café,Factory,Food Court,Food & Drink Shop,Food
3,Central Singapore,Jalan Kayu,1.3972,103.8719,2,Pizza Place,Bar,Noodle House,Dessert Shop,Cupcake Shop,Bus Station,Bus Line,Southern / Soul Food Restaurant,Steakhouse,Hong Kong Restaurant
4,Central Singapore,Bukit Sembawang Estate,1.3956,103.8692,2,Asian Restaurant,Pizza Place,Thai Restaurant,Cupcake Shop,Bus Stop,Southern / Soul Food Restaurant,Bar,Chinese Restaurant,Gourmet Shop,Dessert Shop
5,North East,Kampong Serangoon Kechil,1.3900,103.9072,2,Fast Food Restaurant,Food Stand,Basketball Court,Shoe Store,Coffee Shop,Japanese Restaurant,Sandwich Place,Food Court,Food & Drink Shop,Food
6,South West,Kampong Cutforth,1.3819,103.7364,0,Golf Course,Pool,Restaurant,Beer Garden,Café,Chinese Restaurant,Coffee Shop,Bistro,Gym Pool,Bowling Alley
7,North West,Mok Peng Hiang Estate,1.4456,103.7969,3,Food Court,Trail,Fast Food Restaurant,Yoga Studio,Factory,Food Stand,Food & Drink Shop,Food,Flower Shop,Fishing Spot
8,North West,Woodlands New Town,1.4363,103.7867,2,Japanese Restaurant,Coffee Shop,Clothing Store,Café,Asian Restaurant,Frozen Yogurt Shop,Indian Restaurant,Shopping Mall,Electronics Store,Chinese Restaurant
9,Central Singapore,Kampong Pinang,1.3811,103.8942,2,Metro Station,Wings Joint,Pharmacy,Dim Sum Restaurant,Coffee Shop,Chinese Restaurant,Grocery Store,Residential Building (Apartment / Condo),Fast Food Restaurant,Farm


In [25]:
# Get geographical coordinates of Singapore.
address = 'Singapore'

geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Singapore_merged['Latitude'], Singapore_merged['Longitude'], Singapore_merged['Neighborhood'], Singapore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=[rainbow[cluster-3]],
        fill=True,
        fill_color=rainbow[cluster-3],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Go to https://nbviewer.jupyter.org/github/alvonchua/Coursera_Capstone/blob/main/Week%204%20%26%205%20Assignment_Notebook.ipynb if the map is not rendered

### Cluster 0 Analysis

In [26]:
Cluster_0 = Singapore_merged.loc[Singapore_merged['Cluster Labels'] == 0, Singapore_merged.columns[[1] + list(range(5, Singapore_merged.shape[1]))]]
Cluster_0

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Kampong Cutforth,Golf Course,Pool,Restaurant,Beer Garden,Café,Chinese Restaurant,Coffee Shop,Bistro,Gym Pool,Bowling Alley
11,Kampong Sungai Tengah,Breakfast Spot,Light Rail Station,Supermarket,Shopping Mall,Bus Station,Indonesian Restaurant,Bus Line,BBQ Joint,Food Court,Diner
18,Tay Keng Loon Estate,Bus Station,Factory,Food Court,Indian Restaurant,Supermarket,Farm,Food Stand,Food & Drink Shop,Food,Flower Shop
21,Kampong Beremban,Pet Store,Resort,Farm,Yoga Studio,Electronics Store,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot
33,Kampong Bukit Panjang,Bus Station,Supermarket,Karaoke Bar,Chinese Breakfast Place,Bubble Tea Shop,Chinese Restaurant,Thai Restaurant,Indonesian Restaurant,Plaza,Housing Development
36,Yio Chu Kang Estate,Restaurant,Hotel Pool,Pool,Seafood Restaurant,Bus Station,Roof Deck,Yoga Studio,Food,Flower Shop,Fishing Spot
37,Yio Chu Kang,Bus Station,Restaurant,Pool,Seafood Restaurant,Bus Line,Yoga Studio,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
39,Kampong Amoy Quee,Pool,Food Court,Tennis Court,Fast Food Restaurant,Theater,Gym / Fitness Center,College Auditorium,Dance Studio,Café,Food
40,Singapore United Plantation,Pool,College Cafeteria,BBQ Joint,Sandwich Place,Restaurant,Food Court,Bakery,Pool Hall,Cosmetics Shop,Dessert Shop
46,Saint Michael’s Estate,Café,Soccer Stadium,Coffee Shop,Gym / Fitness Center,Bus Station,Residential Building (Apartment / Condo),Yoga Studio,Food & Drink Shop,Food,Flower Shop


In [27]:
Cluster_0_grouped_first = Cluster_0.groupby('1st Most Common Venue').count()

Cluster_0_grouped_second = Cluster_0.groupby('2nd Most Common Venue').count()

Cluster_0_grouped_third = Cluster_0.groupby('3rd Most Common Venue').count()

Cluster_0_reset1 = Cluster_0_grouped_first[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)
Cluster_0_reset2 = Cluster_0_grouped_second[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)
Cluster_0_reset3 = Cluster_0_grouped_third[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)

pd.concat([Cluster_0_reset1, Cluster_0_reset2, Cluster_0_reset3], axis =1)

,1st Most Common Venue,Neighborhood,2nd Most Common Venue,Neighborhood,3rd Most Common Venue,Neighborhood
0,Bus Station,4.0,Steakhouse,2,Pool,2.0
1,Baseball Stadium,2.0,Pool,2,Factory,2.0
2,Hotel,2.0,Pier,2,Food & Drink Shop,2.0
3,Pool,2.0,Building,1,Office,2.0
4,Breakfast Spot,1.0,Bus Line,1,BBQ Joint,1.0
5,Café,1.0,Soccer Stadium,1,Moving Target,1.0
6,Coffee Shop,1.0,Restaurant,1,Supermarket,1.0
7,Golf Course,1.0,Resort,1,Restaurant,1.0
8,High School,1.0,Planetarium,1,Playground,1.0
9,Military Base,1.0,Light Rail Station,1,Food Court,1.0


### Cluster 1 Analysis

In [28]:
Cluster_1 = Singapore_merged.loc[Singapore_merged['Cluster Labels'] == 1, Singapore_merged.columns[[1] + list(range(5, Singapore_merged.shape[1]))]]
Cluster_1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Kampong Sungai Jurong,Bus Line,Yoga Studio,Cupcake Shop,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot


In [29]:
Cluster_1_grouped_first = Cluster_1.groupby('1st Most Common Venue').count()

Cluster_1_grouped_second = Cluster_1.groupby('2nd Most Common Venue').count()

Cluster_1_grouped_third = Cluster_1.groupby('3rd Most Common Venue').count()

Cluster_1_reset1 = Cluster_1_grouped_first[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)
Cluster_1_reset2 = Cluster_1_grouped_second[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)
Cluster_1_reset3 = Cluster_1_grouped_third[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)

pd.concat([Cluster_1_reset1, Cluster_1_reset2, Cluster_1_reset3], axis =1)

,1st Most Common Venue,Neighborhood,2nd Most Common Venue,Neighborhood,3rd Most Common Venue,Neighborhood
0,Bus Line,1,Yoga Studio,1,Cupcake Shop,1


### Cluster 2 Analysis

In [30]:
Cluster_2 = Singapore_merged.loc[Singapore_merged['Cluster Labels'] == 2, Singapore_merged.columns[[1] + list(range(5, Singapore_merged.shape[1]))]]
Cluster_2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Choa Chu Kang New Town,Coffee Shop,Road,Paper / Office Supplies Store,Fast Food Restaurant,Gym,Grocery Store,Café,Yoga Studio,Food & Drink Shop,Food
2,Bukit Panjang Estate,Coffee Shop,Convenience Store,Fast Food Restaurant,Grocery Store,Supermarket,Café,Factory,Food Court,Food & Drink Shop,Food
3,Jalan Kayu,Pizza Place,Bar,Noodle House,Dessert Shop,Cupcake Shop,Bus Station,Bus Line,Southern / Soul Food Restaurant,Steakhouse,Hong Kong Restaurant
4,Bukit Sembawang Estate,Asian Restaurant,Pizza Place,Thai Restaurant,Cupcake Shop,Bus Stop,Southern / Soul Food Restaurant,Bar,Chinese Restaurant,Gourmet Shop,Dessert Shop
5,Kampong Serangoon Kechil,Fast Food Restaurant,Food Stand,Basketball Court,Shoe Store,Coffee Shop,Japanese Restaurant,Sandwich Place,Food Court,Food & Drink Shop,Food
8,Woodlands New Town,Japanese Restaurant,Coffee Shop,Clothing Store,Café,Asian Restaurant,Frozen Yogurt Shop,Indian Restaurant,Shopping Mall,Electronics Store,Chinese Restaurant
9,Kampong Pinang,Metro Station,Wings Joint,Pharmacy,Dim Sum Restaurant,Coffee Shop,Chinese Restaurant,Grocery Store,Residential Building (Apartment / Condo),Fast Food Restaurant,Farm
10,Serangoon,Chinese Restaurant,Park,Playground,Shopping Mall,Bus Station,Supermarket,Coffee Shop,Food Court,Vegetarian / Vegan Restaurant,Convenience Store
12,Kampong Tongkang Pechah,Asian Restaurant,Food Court,Fast Food Restaurant,Supermarket,Coffee Shop,Soup Place,Café,Japanese Restaurant,Sushi Restaurant,Indian Restaurant
14,Kangkar,Convenience Store,Park,Fried Chicken Joint,Food Court,Asian Restaurant,Vegetarian / Vegan Restaurant,Trail,Big Box Store,Laundromat,Ice Cream Shop


In [31]:
Cluster_2_grouped_first = Cluster_2.groupby('1st Most Common Venue').count()

Cluster_2_grouped_second = Cluster_2.groupby('2nd Most Common Venue').count()

Cluster_2_grouped_third = Cluster_2.groupby('3rd Most Common Venue').count()

Cluster_2_reset1 = Cluster_2_grouped_first[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)
Cluster_2_reset2 = Cluster_2_grouped_second[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)
Cluster_2_reset3 = Cluster_2_grouped_third[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)

pd.concat([Cluster_2_reset1, Cluster_2_reset2, Cluster_2_reset3], axis =1)

,1st Most Common Venue,Neighborhood,2nd Most Common Venue,Neighborhood,3rd Most Common Venue,Neighborhood
0,Chinese Restaurant,10.0,Coffee Shop,5.0,Chinese Restaurant,3
1,Coffee Shop,6.0,Park,4.0,Coffee Shop,3
2,Asian Restaurant,4.0,Asian Restaurant,3.0,Fast Food Restaurant,3
3,Café,3.0,Pizza Place,3.0,Food Court,3
4,Food Court,3.0,Bus Station,2.0,Sushi Restaurant,2
5,Pizza Place,3.0,Convenience Store,2.0,Café,2
6,Fast Food Restaurant,2.0,Food Court,2.0,Park,2
7,Accessories Store,1.0,Grocery Store,1.0,Noodle House,2
8,Bar,1.0,Thai Restaurant,1.0,Asian Restaurant,1
9,Bridge,1.0,Shopping Mall,1.0,Music Venue,1


### Cluster 3 Analysis

In [32]:
Cluster_3 = Singapore_merged.loc[Singapore_merged['Cluster Labels'] == 3, Singapore_merged.columns[[1] + list(range(5, Singapore_merged.shape[1]))]]
Cluster_3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Keat Hong Village,Miscellaneous Shop,Food Court,Coffee Shop,Bus Line,Fast Food Restaurant,Kids Store,Yoga Studio,Farmers Market,Food Stand,Food & Drink Shop
7,Mok Peng Hiang Estate,Food Court,Trail,Fast Food Restaurant,Yoga Studio,Factory,Food Stand,Food & Drink Shop,Food,Flower Shop,Fishing Spot
13,Kampong Teban,Food Court,Ice Cream Shop,Laundromat,Big Box Store,New American Restaurant,Yoga Studio,Farm,Food Stand,Food & Drink Shop,Food
22,Marsiling,Food Court,Park,Bus Station,Gift Shop,Yoga Studio,Farm,Food & Drink Shop,Food,Flower Shop,Fishing Spot
24,Somapah Serangoon,Food Court,Asian Restaurant,Stadium,Yoga Studio,Factory,Food Stand,Food & Drink Shop,Food,Flower Shop,Fishing Spot
31,Kampong Sireh,American Restaurant,Dessert Shop,Food Court,Park,Ice Cream Shop,Yoga Studio,Farm,Food Stand,Food & Drink Shop,Food
38,Ang Mo Kio New Town,Food Court,Park,Australian Restaurant,Shanghai Restaurant,Bus Stop,Chinese Restaurant,Thai Restaurant,Coffee Shop,Malay Restaurant,Vegetarian / Vegan Restaurant
41,Kampong Ladang,Food Court,Coffee Shop,Chinese Restaurant,Food,Motorcycle Shop,Supermarket,Restaurant,Diner,Discount Store,Food & Drink Shop
43,Kampong Reteh,Food Court,Coffee Shop,Chinese Restaurant,Food,Motorcycle Shop,Supermarket,Restaurant,Diner,Discount Store,Food & Drink Shop
52,Kampong Pengkalan Petai,Warehouse Store,Food Court,Eastern European Restaurant,Yoga Studio,Factory,Food Stand,Food & Drink Shop,Food,Flower Shop,Fishing Spot


In [33]:
Cluster_3_grouped_first = Cluster_3.groupby('1st Most Common Venue').count()

Cluster_3_grouped_second = Cluster_3.groupby('2nd Most Common Venue').count()

Cluster_3_grouped_third = Cluster_3.groupby('3rd Most Common Venue').count()

Cluster_3_reset1 = Cluster_3_grouped_first[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)
Cluster_3_reset2 = Cluster_3_grouped_second[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)
Cluster_3_reset3 = Cluster_3_grouped_third[['Neighborhood']].sort_values(by='Neighborhood', ascending = False).reset_index(drop=False)

pd.concat([Cluster_3_reset1, Cluster_3_reset2, Cluster_3_reset3], axis =1)

,1st Most Common Venue,Neighborhood,2nd Most Common Venue,Neighborhood,3rd Most Common Venue,Neighborhood
0,Food Court,7.0,Food Court,4.0,Chinese Restaurant,2
1,American Restaurant,1.0,Coffee Shop,2.0,Coffee Shop,2
2,Coffee Shop,1.0,Park,2.0,Australian Restaurant,1
3,Convenience Store,1.0,Asian Restaurant,1.0,Bus Station,1
4,Miscellaneous Shop,1.0,Dessert Shop,1.0,Convenience Store,1
5,Warehouse Store,1.0,Ice Cream Shop,1.0,Eastern European Restaurant,1
6,NaN,NaN,Trail,1.0,Fast Food Restaurant,1
7,NaN,NaN,NaN,NaN,Food Court,1
8,NaN,NaN,NaN,NaN,Laundromat,1
9,NaN,NaN,NaN,NaN,Stadium,1
